In [1]:
'''Here are all the modules we will be using in this notebook. Make sure you can import everything before moving on!'''

import numpy as np
import pandas as pd
import tensorflow as tf

import sys
import os
sys.path.append(os.path.abspath('./src'))
from model import *
from transformer import *
from observer import *

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display

data_root = './data/'

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
raw_df = pd.read_hdf(data_root + 'train.h5')

# raw_sample_df = raw_df[raw_df['id'].isin([2047,1738])]
raw_sample_df = raw_df[raw_df['id'].isin([2047])]
raw_df = None # Release memory

In [3]:
raw_sample_df.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
708,2047,0,-0.195484,-0.019851,1.125294,-0.225663,1.276203,-0.087181,-0.094974,0.508488,...,-0.526171,NaN,NaN,NaN,-0.240367,NaN,NaN,-2.0,NaN,0.017816
1458,2047,1,-0.195484,-0.019851,1.125294,-0.223219,1.276203,-0.087650,-0.094974,0.501267,...,-0.503224,NaN,NaN,NaN,-0.239247,NaN,NaN,-2.0,NaN,-0.009593
2208,2047,2,-0.195484,-0.019851,1.125294,-0.222246,1.276203,-0.087837,-0.094974,0.498390,...,-0.494082,NaN,NaN,NaN,-0.238801,NaN,NaN,-2.0,NaN,0.021994
2958,2047,3,-0.195484,-0.019851,1.125294,-0.221406,1.276203,-0.087999,-0.094974,0.495908,...,-0.486194,NaN,NaN,NaN,-0.238416,NaN,NaN,-2.0,NaN,0.029161
3708,2047,4,-0.195484,-0.019851,1.125294,-0.220053,1.276203,-0.088258,-0.094974,0.491910,...,-0.473489,NaN,NaN,NaN,-0.237796,NaN,NaN,-2.0,NaN,-0.006530


In [4]:
raw_sample_df.shape

(1813, 111)

In [5]:
train_prop = 0.66
num_obs = raw_sample_df.shape[0]

train_df = raw_sample_df[:int(num_obs*train_prop)]

In [6]:
transf = Transformer()
transf.fit(train_df)

scaled_train_df = transf.transform(train_df)

/usr/local/lib/python2.7/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
scaled_train_df.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
708,2047,0,-0.595979,-0.33508,-0.182061,0.037455,1.776757,-0.599295,-0.110482,1.968132,...,-0.630396,0.469681,0.772965,0.779589,-1.359359,0.071677,0.210758,-1.672128,-0.157704,0.017816
1458,2047,1,-0.595979,-0.33508,-0.182061,0.066503,1.776757,-0.604289,-0.110482,1.950626,...,-0.605083,0.469681,0.772965,0.779589,-1.356480,0.071677,0.210758,-1.672128,-0.157704,-0.009593
2208,2047,2,-0.595979,-0.33508,-0.182061,0.078076,1.776757,-0.606279,-0.110482,1.943651,...,-0.594998,0.469681,0.772965,0.779589,-1.355333,0.071677,0.210758,-1.672128,-0.157704,0.021994
2958,2047,3,-0.595979,-0.33508,-0.182061,0.088060,1.776757,-0.607996,-0.110482,1.937633,...,-0.586297,0.469681,0.772965,0.779589,-1.354343,0.071677,0.210758,-1.672128,-0.157704,0.029161
3708,2047,4,-0.595979,-0.33508,-0.182061,0.104143,1.776757,-0.610761,-0.110482,1.927940,...,-0.572282,0.469681,0.772965,0.779589,-1.352749,0.071677,0.210758,-1.672128,-0.157704,-0.006530


In [8]:
num_epoch = 400
batch_size = 1
num_feat = 1
lstm_size = 32
fc_hidd_size = 16
lr = 1e-3

M = Model(num_feat=num_feat, lstm_size=lstm_size, fc_hidd_size=fc_hidd_size, lr=lr)
session = tf.Session()
session.run(tf.global_variables_initializer())

In [9]:
history = []
for loss in M.fit(session, scaled_train_df, num_epoch, batch_size):
    history.append(loss)
    dyn_plot(np.sqrt(history))

ValueError: Cannot feed value of shape (1, 1196, 109) for Tensor u'X:0', which has shape '(?, ?, 1)'